In [3]:
df = pd.read_csv('Algerian_forest_fires_dataset_UPDATE.csv', header=1)

NameError: name 'pd' is not defined

In [ ]:
df.head()

In [ ]:
df[df.isnull().any(axis=1)]

In [ ]:
df.loc[:122,"Region"]=0
df.loc[122:,"Region"]=1
df[["Region"]] = df[["Region"]].astype(int)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()
df = df.dropna().reset_index(drop=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.iloc[[122]]
df = df.drop(122).reset_index(drop=True)

In [ ]:
df.columns = df.columns.str.strip()

In [ ]:
df.columns

In [ ]:
df.iloc[[122]]

In [ ]:
df.info()

In [ ]:
df[['day','month','year','Temperature','RH','Ws']] = df[['day','month','year','Temperature','RH','Ws']].astype(int)

In [ ]:
df.info()

In [ ]:

df[['Rain','FFMC','DMC','DC','ISI','BUI','FWI']] = df[['Rain','FFMC','DMC','DC','ISI','BUI','FWI']].astype(float)

In [ ]:
df.info()

In [ ]:
df.to_csv('Forest_Fire_Cleaned.csv',index=False)

In [ ]:
df = pd.read_csv('Forest_Fire_Cleaned.csv')

In [ ]:
df = df.drop(['day','month','year'],axis=1)
df_int = df.copy()


In [ ]:
df

In [ ]:

df['Classes'].value_counts()

In [ ]:
df['Classes']= np.where(df['Classes'].str.contains('not fire'),'not fire','fire')
df['Classes'].value_counts()

In [ ]:
df_int['Classes']= np.where(df_int['Classes'].str.contains('not fire'),0,1)


In [ ]:
df_int['Classes'].value_counts()

In [ ]:

df.hist(bins=50,figsize=(20,15))
plt.show()

In [ ]:
df_int.corr()

In [ ]:
sns.heatmap(df_int.corr())

## Model Training


In [ ]:
df_int['Classes'].value_counts()

In [ ]:
df['Classes'].value_counts()

In [ ]:
df.iloc[[122]]

In [ ]:
dff = df_int
dff

In [ ]:
X = dff.drop('FWI',axis=1)
y = dff['FWI']

In [ ]:
y.head()

In [ ]:
X.head()

## Train Test Split for model

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25, random_state=42)

In [ ]:
X_train.corr()

In [ ]:
def correlation(dataset, threshold):
    col_corr = set()
    corr_matrix  = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i,j])> threshold:
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

In [ ]:
corr_features = correlation(X_train,0.85)


In [ ]:
corr_features

In [ ]:
X_train.drop(corr_features,axis=1,inplace=True)
X_test.drop(corr_features,axis=1,inplace=True)
X_train.shape,X_test.shape

## Standardisation

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [ ]:
X_train_scaled

## Box Plots To understand Effect Of Standard Scaler

In [ ]:

plt.subplots(figsize=(15, 5))
plt.subplot(1, 2, 1)
sns.boxplot(data=X_train)
plt.title('X_train Before Scaling')
plt.subplot(1, 2, 2)
sns.boxplot(data=X_train_scaled)
plt.title('X_train After Scaling')

## Linear Regression Model

In [ ]:

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
linreg=LinearRegression()
linreg.fit(X_train_scaled,y_train)
y_pred=linreg.predict(X_test_scaled)
mae=mean_absolute_error(y_test,y_pred)
score=r2_score(y_test,y_pred)
print("Mean absolute error", mae)
print("R2 Score", score)
plt.scatter(y_test,y_pred)

## Lasso Regression

In [ ]:

from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
lasso=Lasso()
lasso.fit(X_train_scaled,y_train)
y_pred=lasso.predict(X_test_scaled)
mae=mean_absolute_error(y_test,y_pred)
score=r2_score(y_test,y_pred)
print("Mean absolute error", mae)
print("R2 Score", score)
plt.scatter(y_test,y_pred)

Cross Validation Lasso

In [ ]:

from sklearn.linear_model import LassoCV
lassocv=LassoCV(cv=5)
lassocv.fit(X_train_scaled,y_train)

In [ ]:
y_pred=lassocv.predict(X_test_scaled)
plt.scatter(y_test,y_pred)
mae=mean_absolute_error(y_test,y_pred)
score=r2_score(y_test,y_pred)
print("Mean absolute error", mae)
print("R2 Score", score)

## Ridge Regression model

In [ ]:

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
ridge=Ridge()
ridge.fit(X_train_scaled,y_train)
y_pred=ridge.predict(X_test_scaled)
mae=mean_absolute_error(y_test,y_pred)
score=r2_score(y_test,y_pred)
print("Mean absolute error", mae)
print("R2 Score", score)
plt.scatter(y_test,y_pred)

In [ ]:
from sklearn.linear_model import RidgeCV
ridgecv=RidgeCV(cv=5)
ridgecv.fit(X_train_scaled,y_train)
y_pred=ridgecv.predict(X_test_scaled)
plt.scatter(y_test,y_pred)
mae=mean_absolute_error(y_test,y_pred)
score=r2_score(y_test,y_pred)
print("Mean absolute error", mae)
print("R2 Score", score)

In [ ]:
ridgecv.get_params()

## Elasticnet Regression

In [ ]:

from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
elastic=ElasticNet()
elastic.fit(X_train_scaled,y_train)
y_pred=elastic.predict(X_test_scaled)
mae=mean_absolute_error(y_test,y_pred)
score=r2_score(y_test,y_pred)
print("Mean absolute error", mae)
print("R2 Score", score)
plt.scatter(y_test,y_pred)

In [ ]:
from sklearn.linear_model import ElasticNetCV
elasticcv=ElasticNetCV(cv=5)
elasticcv.fit(X_train_scaled,y_train)
y_pred=elasticcv.predict(X_test_scaled)
plt.scatter(y_test,y_pred)
mae=mean_absolute_error(y_test,y_pred)
score=r2_score(y_test,y_pred)
print("Mean absolute error", mae)
print("R2 Score", score)

## Pickle Models
ML Model &
Preprocessing Standard Scaler

In [ ]:
scaler

In [ ]:
ridge

In [ ]:
import pickle
pickle.dump(scaler,open('scaler.pkl','wb'))
pickle.dump(linreg,open('linreg.pkl','wb'))
pickle.dump(ridge,open('ridge.pkl','wb'))